# Projet Optimisation (groupe 6) - Benjamin Piet & Damien Capéraa

## 2. Etude et resolution numerique

### Question 1

Notre problème d'optimisation est le suivant : 
$$\min_{C^Tp - d}\frac{1}{2}p^TAp-b^Tp$$\
On note $f(p)=\frac{1}{2}p^TAp-b^Tp$ la fonction coût et $c(p)=C^Tp-d$ les contraintes.
Dans la suite, on notera $x = p$ pour éviter toute confusion entre la variable des prix $p$ et les directions de descente.

##### Etude du problème
La fonction f est **fortement convexe** car la matrice $A$ est symétrique definie positive. Pour le démontrer, on peut déjà remarquer que $A$ est symétrique puis utiliser le théorème d'éuivalence $A \in S_n^{++}(\mathbb{R}) \Leftrightarrow A \in S_n(\mathbb{R}) \, et \, Sp(A) \subset \mathbb{R}^{+*}$ et calculer les valeurs propres de A avec Python :

In [3]:
import numpy as np
A = np.array([[3.0825, 0, 0, 0], [0, 0.0405, 0, 0], [0, 0, 0.0271, -0.0031], \
     [0, 0, -0.0031, 0.0054]])
valeurs_propres = np.linalg.eig(A)[0]
print(f"Liste des valeurs propres de A : {valeurs_propres}")

Liste des valeurs propres de A : [0.02753417 0.00496583 3.0825     0.0405    ]


Toutes les valeurs propres de $A$ étant strictement positives, elle est symétrique définie positive. Par conséquent, la fonction de coût quadratique f est convexe.\
De plus, l'ensemble de recherche est convexe, car les contraintes sont toutes affines.
On a donc **l'existence d'un unique minimum global**.\
\
Le conditionnement du problème vaut :
$$K(A) = \frac{max(Sp(A))}{min(Sp(A))} = 620,7$$
***Faire des recherches sur le conditionnement***

##### Choix d'une méthode de résolution
* Les contraintes de notre problème sont des contraintes d'inégalités. Nous proposons d'utiliser un algorithme de recherche du point selle du Lagrangien. En effet, un point $(x^*, \lambda ^*)$ est un point selle du Lagrangien si et seulement si $x^*$ est solution du problème d'otpimisation.
* En outre, si l'on suppose que les contraintes actives en $x^*$ sont qualifiées, alors la convexite de $f$ garantit l'existence d'un point selle du Lagrangien grâce au théorème 29 du cours.
* Finalement, nous proposons d'implémenter **l'algorithme d'Uzawa**, qui repose bien sur le concept de dualité du problème (p.38 du polycopié) : à partir de $x^0 \in \mathbb{R}^4$, $\lambda^0 \in \mathbb{R}^7$, $\rho \in \mathbb{R}^+$ quelconques, on note $\mathbb{R}^7 \ni \lambda \mapsto P(\lambda) \in (\mathbb{R}^+)^7$ la projection sur $(\mathbb{R}^+)^7$, itérer : 
    * résoudre $\min_{x \in \mathbb{R}^4} \mathcal{L}(x, \lambda ^k)$, on note $x^{k+1}$ la solution ;
    * $\lambda ^{k+1} = P(\lambda ^k + \rho c(x^{k+1}))$.
* Pour la partie minimisation, nous allons utiliser un algorithme de gradient à pas optimal, en déterminant un pas à chaque itération à l'aide des conditions de Wolfe.

### Question 2

   En fait, après avoir testé avec la fonction scipy.optimize pour avoir une idée de la valeur attendu, nous avons trouvé que l'algorithme à pas fixe semble mieux converger que celui à pas de Wolfe, et en moins de temps (même si le nombre d'iteration est bien plus grand, le temps d'execution est plus faible).\
   Nous mettons tous de même les deux algorithmes, mais nous vous conseillons de ne pas lancer celui à pas de Wolfe

In [4]:
#Mise en place des fonctions et valeurs numÃ©riques
import numpy as np

A = np.array([[3.0825, 0, 0, 0], [0, 0.0405, 0, 0], [0, 0, 0.0271, -0.0031], \
	[0, 0, -0.0031, 0.0054]])
b = np.array([[2671], [135], [103], [19]])
C = np.array([[-0.0401, -0.1326, 1.5413, 0.0, 0.0, 0.0, 0.0160], \
	[-0.0162, -0.0004, 0.0, 0.0203, 0.0, 0.0, 0.0004], \
	[-0.0039, -0.0034, 0.0, 0.0, 0.0136, -0.0016, 0.0005], \
	[0.0002, 0.0006, 0.0, 0.0, -0.0015, 0.0027, 0.0002]])
d = np.array([[-92.6], [-29.0], [2671], [135], [103], [19], [10]])
eps = 1e-8 # paramÃ¨tre donnant la valeur maximale des composantes
# de la direction pk pour que pk ne soit pas considÃ©rÃ©e comme nulle.

def f(p):
	return float(0.5 * np.matmul(p.T, np.matmul(A, p)) - np.dot(b.T, p))

def c(p):
	return np.matmul(C.T, p) - d

def grad_f(p):
	return np.matmul(A, p) - b

def grad_c(p):
	return C

x0 = np.ones((4, 1))
lambda0 = 100*np.ones((7, 1))

In [5]:
def uzawa_fixed_step(fun, grad_fun, c, grad_c, x0, l, rho, lambda0, max_iter = 10000000, epsilon_grad_L = 1e-8):
	k = 0
	xk = x0
	lambdak = lambda0
	grad_Lagrangien_xk = grad_fun(xk) + np.matmul(grad_c(xk), lambdak)
	while ((k < max_iter) and (np.linalg.norm(grad_Lagrangien_xk) > epsilon_grad_L)):
		grad_Lagrangien_xk = grad_fun(xk) + np.matmul(grad_c(xk), lambdak)
		pk = -grad_Lagrangien_xk
		xk = xk + l*pk
		lambdak = np.array([[max(0, lambdak[i, 0] + rho*c(xk)[i, 0])] for i in range(7)]) # projection sur R+^7
		k = k + 1
	print("Nombre d'iterations : ", k)
	print("lambdak : ", lambdak)
	return xk

print("Uzawa fixed step...")
#Note: après avoir testé l'algo avec conditions de wolfe, on a un pas au début 
#qui est au alentours de 0.5, puis arrive vers 4 à la fin. 
#C'est pour cela que nous avons choisi un pas de 0.5 pour l'ago à pas fixe
x_fixed_step = uzawa_fixed_step(f, grad_f, c, grad_c, x0, 0.5, 0.1, lambda0)

print("x_fixed_step : ", x_fixed_step)
print("c(x_fixed_step) : ", c(x_fixed_step))

Uzawa fixed step...
Nombre d'iterations :  2002225
lambdak :  [[ 4106.17888248]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [96268.16780048]]
x_fixed_step :  [[ 420.23263208]
 [4025.00816724]
 [2774.95779496]
 [1393.98145128]]
c(x_fixed_step) :  [[ 3.43035822e-08]
 [-3.69313179e+01]
 [-2.02329544e+03]
 [-5.32923342e+01]
 [-6.73515462e+01]
 [-1.96761826e+01]
 [ 5.67878631e-07]]


In [ ]:
def wolfe_step(fun, grad_fun, xk, pk, c1 = 0.25, c2 = 0.75, M = 1000):
	l_moins = 0
	l_plus = 0
	f_xk = fun(xk)
	grad_f_xk = grad_fun(xk)
	li = 0.001
	i = 0
	while(i < M):
		if (fun(xk + li*pk) > (f_xk + c1*li*np.dot(grad_f_xk.T, pk))):
			l_plus = li
			li = (l_moins + l_plus)/2.0
		else:
			if (np.dot(grad_fun(xk + li*pk).T, pk) < c2*np.dot(grad_f_xk.T, pk)):
				l_moins = li
				if (l_plus == 0):
					li = 2*li
				else:
					li = (l_moins + l_plus)/2.0
			else:
				return li
		i = i + 1
	return li


def uzawa_wolfe_step(fun, grad_fun, c, grad_c, x0, rho, lambda0, max_iter = 1000000, epsilon_grad_L = 1e-3):
	k = 0
	xk = x0
	lambdak = lambda0
	grad_Lagrangienk_xk = grad_fun(xk) + np.matmul(grad_c(xk), lambdak)
	while ((k < max_iter) and (np.linalg.norm(grad_Lagrangienk_xk) > epsilon_grad_L)):
		Lagrangienk = lambda x : fun(x) + np.dot(lambdak.T, c(x))
		grad_Lagrangienk = lambda x : grad_fun(x) + np.matmul(grad_c(x), lambdak)
		grad_Lagrangienk_xk = grad_Lagrangienk(xk)
		pk = -grad_Lagrangienk_xk
		lk = wolfe_step(Lagrangienk, grad_Lagrangienk, xk, pk)
		xk = xk + lk*pk
		lambdak = np.array([[max(0, lambdak[i, 0] + rho*c(xk)[i, 0])] for i in range(7)]) # projection sur R+^7
		k = k + 1
	print("Nombre d'iterations : ", k)
	print("lk : ", lk)
	return xk

print(uzawa_wolfe_step(f, grad_f, c, grad_c, x0, 0.1, lambda0))

In [ ]:
#scipy.optimize

On trouve donc comme prix en euros/tonnes optimal de :
* ***420*** pour le ***lait***
* ***4025*** pour le ***beurre***
* ***2775*** pour le ***gouda***
* ***1394*** pour le ***edam***

Cela fait donc comme quantité:
* 2023 tonnes pour le lait
* 53.3 tonnes pour le beurre
* 67.4 tonnes pour le gouda
* 19.9 tonnes pour l'edam

Notes: nous ne sommes pas totalement sur au niveau des unités

## 3. Etude avancée

### Question 1

Tout d'abord, au vu des resultats, on peut supposer que seuls les contraintes 1 et 7 sont actives. Ce qui donne alors comme équations (en annulant le gradient du lagrangien)
$$3.0825p_1-0.0401\lambda_1+0.016\lambda_7+2671=0$$
$$0.0405p_2-0.0162\lambda_1+0.0004\lambda_7+135=0$$
$$0.0271p_3-0.0031p_4-0.0039\lambda_1+0.0005\lambda_7+103$$
$$-0.0031p_3+0.0054p_4+0.0002\lambda_1+0.0002\lambda_7+19=0$$
$$0.0401p_1+0.0162p_2+0.0039p_3-0.0002p_4=92.6$$
$$0.0160p_1+0.0004p_2+0.0005p_3+0.0002p_4=10$$

Il suffit alors de résoudre ce système. On peut pour cela utiliser python qui facilitera notre tache.

## Annexe

Nous avons, dans la partie 2, essayé dans un premier temps de developper un algorithme des contraintes actives pour résoudre le problème. Nous mettons ici ce que nous avons accompli, même si il n'y a pas convergence.

**Etape 2 (a) de l'algorithme : résolution du problème d'otpimisation sous contraintes d'égalité** \
L'étape consiste à résoudre $\displaystyle \min_{c_i^Tp = 0, i \in W^k} \frac{1}{2}p^TGp + (Gx^k + d)^Tp$.
On introduit le Lagrangien du problème $\mathcal{L}(p, \lambda) = \frac{1}{2}p^TGp + (Gx^k + d)^Tp + \lambda^TD^Tp$ ou $D$ est la matrice composée des colonnes $c_i$ de C pour $i \in W^k$.
Un point stationnaire du Lagrangien sans contraintes est un point stationnaire de la fonction coût sous contraintes : on cherche donc le point $(p^*, \lambda^*)$ qui annule les deux dérivées partielles de $\mathcal{L}$.
$$
\begin{align}
\frac{\partial\mathcal{L}}{\partial\lambda}(p^*, \lambda^*) &= 0 = D^Tp \\
\frac{\partial\mathcal{L}}{\partial p}(p^*, \lambda^*) &= 0 = Ap + (Ax^k + b) + D\lambda \\
\end{align}$$
On réécrit le problème sous forme matricielle car c'est la formulation qui peut être exploitée avec la fonction numpy `np.linalg.solve`.
$$
\begin{pmatrix} D^T&0 \\
A&D\\
\end{pmatrix}
\begin{pmatrix} p \\
\lambda \\
\end{pmatrix}
=
\begin{pmatrix} 0 \\
-Ax^k - b \\
\end{pmatrix}
$$

In [1]:
#Algorithme des contraintes actives
import numpy as np

A = np.array([[3.0825, 0, 0, 0], [0, 0.0405, 0, 0], [0, 0, 0.0271, -0.0031], \
     [0, 0, -0.0031, 0.0054]])
b = np.array([[2671], [135], [103], [19]])
C = np.array([[-0.0401, -0.0162, -0.0039, 0.0002], \
    [-0.1326, -0.0004, -0.0034, 0.0006], [1.5413, 0, 0, 0], \
    [0, 0.0203, 0, 0], [0, 0, 0.0136, -0.0015], \
    [0, 0, -0.0016, 0.0027], [0.0160, 0.0004, 0.0005, 0.0002]]).transpose()
d = np.array([[-92.6], [-29.0], [2671], [135], [103], [19], [10]])
eps = 1e-8 # paramètre donnant la valeur maximale des composantes
# de la direction pk pour que pk ne soit pas considérée comme nulle.

def f(p):
    return float(0.5 * np.matmul(p.transpose(), np.matmul(A, p)) - np.dot(b.transpose(), p))

def c(p):
    return np.matmul(C.transpose(), p) - d


def xk_is_solution(xk, Wk):
    '''
    Etape 1 de l'algorithme des contraintes actives.
    Détermine, grâce aux conditions KKT, si xk est une solution du problème
    d'optimisation restreint aux contraintes contenues dans Wk.
    '''
    CC = C[:, Wk]
    dd = d[Wk, :]
    rg = np.linalg.matrix_rank(CC)
    if rg == len(Wk):
        # Les contraintes actives sont qualifiées
        # Conditions de relâchement
        Wk_systeme = Wk.copy()
        for i, c_i in enumerate([CC[:, j] for j in range(len(Wk))]):
            if np.dot(c_i, xk) -  dd[i, 0] != 0:
                # Lambda_i = 0 : on retire la contrainte Wk_systeme[i]
                del Wk_systeme[Wk_systeme.index(Wk[i])]
        Lambda = np.zeros((len(Wk), 1))
        print(Wk_systeme)
        if len(Wk_systeme) != 0:
            CC_systeme = C[:, Wk_systeme]
            dd_systeme = d[Wk_systeme, :]
            Lambda_systeme = np.linalg.solve(CC_systeme.transpose(), dd_systeme)
            for j in Wk:
                if j in Wk_systeme:
                    Lambda[Wk.index(j), 0] = Lambda_systeme[Wk_systeme.index(j), 0]
        return all([Lambda[i, 0] > 0 for i in range(len(Wk))]), Lambda
    else:
        # Les contraintes actives ne sont pas qualifiées
        return False, None

W0 = [0, 1, 2, 3] # arbitraire mais permet de n'avoir qu'un point de départ possible
# car le système à résoudre contient 4 équations pour 4 inconnues.
# Recherche de p0 où ces 4 contraintes sont actives
CC = C[:, W0]
dd = d[W0, :]
x0 = np.linalg.solve(CC.transpose(), dd)

def contraintes():
    xk = x0
    Wk = W0
    while True:
        if xk_is_solution(xk, Wk)[0]:
            return xk, Wk
        # (a)
        '''
        L'étape (a) consiste à résoudre un problème d'optimisation sous
        contraintes d'égalité. On cherche alors (p*, Lambda*) point stationnaire
        du lagrangien associé. Cette recherche, comme présenté dans le notebook,
        aboutit à la résolution d'un système linéaire.
        '''
        D = C[:, Wk]
        E_ligne_0 = np.concatenate((D.transpose(), np.zeros((D.shape[1], D.shape[1]))), axis=1)
        E_ligne_1 = np.concatenate((A, D), axis=1)
        E = np.concatenate((E_ligne_0, E_ligne_1), axis=0)
        F = np.concatenate((np.zeros((D.shape[1], 1)), -np.matmul(A, xk) + b), axis=0)
        X = np.linalg.solve(E, F)
        pk = X[0:4, :]
        if any([abs(pk[i, 0]) > eps for i in range(4)]):
            # (b) : pk != 0
            W_barre = [i for i in range(7) if i not in Wk]
            L, indices = [], []
            for i in W_barre:
                c_i = C[:,i]
                if np.dot(c_i, pk) > 0:
                    L.append((d[i, 0] - np.dot(c_i, xk))/np.dot(c_i, pk))
                    indices.append(i)
            if L != []:
                alphak = min(1, min(L))
            else:
                alphak = 1
            xk = xk + alphak*pk
            if alphak < 1:
                j = indices[L.index(min(L))]
                Wk.append(j)
        else:
            # (c) ! pk = 0
            booleen, Lambda = xk_is_solution(xk, Wk)
            if booleen:
                return xk, Wk
            Lambda = Lambda.tolist()
            c = Lambda.index(min(Lambda))
            del Wk[c]
    return xk, Wk

In [2]:
contraintes()

[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 3]


LinAlgError: Last 2 dimensions of the array must be square